In [2]:
import random
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.8.0


## Synthetic data

We create a set of *boards* of size $5 \times 5$ with 3 channels for each position. The label at each position $(j,k)$ is computed as a function of the channel values $v_{jkl}$ at that position.

$$
    L_{jk} = \sum_{l=0}^2 a_l \cdot (v_{jkl})^{l+1}
$$

Here, $a_l$ denote arbitrary coefficients defined below. Note that this function is the same for every position. Thus, a sufficiently deep convolutional network with only $1 \times 1$ kernels should easily learn this function by simultaneously looking at all the positions of any given training board.

In [3]:
def create_data(n):
    _batch = np.zeros([n,5,5,3])
    _labels = np.zeros([n,5,5,1])
    _a=[.9, .3, -.2]
    for _i in range(n):
        for _x in range(5):
            for _y in range(5):
                for _l in range(3):
                    _v = 2*(random.random()-0.5)
                    _batch[_i][_x][_y][_l] = _v
                    _labels[_i][_x][_y][0] += _a[_l] * _v**(_l+1)
    return _batch, _labels

In [47]:
N = 1000
train, labels = create_data(N)

In [48]:
train.shape

(1000, 5, 5, 3)

Smartly rearranging the dimensions of the first *board* of the batch shows the three $5 \times 5$ channels

In [49]:
print(np.rollaxis(train[0], 2, 0))

[[[-0.85896927  0.69902529 -0.53124401 -0.20641162  0.61040522]
  [-0.06872278  0.76939992  0.03751382  0.04917618  0.95711085]
  [-0.59620993  0.30264705  0.15305457 -0.82698051 -0.80078748]
  [-0.03169596  0.58075038  0.38751542  0.77750194 -0.68356026]
  [-0.84472966  0.63342431  0.72946307 -0.30480892  0.3689736 ]]

 [[-0.8730583   0.3590971  -0.99766832  0.28599123 -0.77423295]
  [-0.82824754 -0.23720669  0.30697065 -0.74464368 -0.70631168]
  [-0.04318187  0.04302036 -0.14945129  0.59630714 -0.58375218]
  [ 0.77249473 -0.38070552  0.39028454  0.84041692 -0.35273369]
  [-0.76712986  0.8356657   0.46770037 -0.68763423 -0.30861735]]

 [[-0.77026088  0.2825407  -0.19212071  0.72794028  0.24096053]
  [-0.9953683   0.19143189  0.66628494 -0.04527839 -0.08705117]
  [-0.90661841 -0.68658508 -0.57674072 -0.27628564 -0.5172958 ]
  [-0.3305647  -0.20905938  0.97747832  0.93200282 -0.1019259 ]
  [ 0.41382766 -0.9423963   0.75783356 -0.76777396 -0.19900385]]]


## TF 2.8

In [50]:
tf_train = tf.data.Dataset.from_tensor_slices(train)
tf_train

<TensorSliceDataset element_spec=TensorSpec(shape=(5, 5, 3), dtype=tf.float64, name=None)>

In [51]:
tf_labels = tf.data.Dataset.from_tensor_slices(labels)
tf_labels

<TensorSliceDataset element_spec=TensorSpec(shape=(5, 5, 1), dtype=tf.float64, name=None)>

In [52]:
train_input = tf.data.Dataset.zip((tf_train, tf_labels)).batch(10)

In [53]:
train_data, train_labels = next(iter(train_input))

In [54]:
train_data

<tf.Tensor: shape=(10, 5, 5, 3), dtype=float64, numpy=
array([[[[-8.58969271e-01, -8.73058304e-01, -7.70260878e-01],
         [ 6.99025289e-01,  3.59097099e-01,  2.82540698e-01],
         [-5.31244012e-01, -9.97668320e-01, -1.92120712e-01],
         [-2.06411625e-01,  2.85991231e-01,  7.27940278e-01],
         [ 6.10405225e-01, -7.74232950e-01,  2.40960527e-01]],

        [[-6.87227781e-02, -8.28247541e-01, -9.95368299e-01],
         [ 7.69399920e-01, -2.37206694e-01,  1.91431887e-01],
         [ 3.75138201e-02,  3.06970649e-01,  6.66284936e-01],
         [ 4.91761772e-02, -7.44643680e-01, -4.52783929e-02],
         [ 9.57110849e-01, -7.06311684e-01, -8.70511716e-02]],

        [[-5.96209934e-01, -4.31818670e-02, -9.06618405e-01],
         [ 3.02647054e-01,  4.30203645e-02, -6.86585077e-01],
         [ 1.53054573e-01, -1.49451288e-01, -5.76740718e-01],
         [-8.26980514e-01,  5.96307142e-01, -2.76285642e-01],
         [-8.00787482e-01, -5.83752183e-01, -5.17295802e-01]],

        [

In [55]:
train_labels

<tf.Tensor: shape=(10, 5, 5, 1), dtype=float64, numpy=
array([[[[-4.53003667e-01],
         [ 6.63296976e-01],
         [-1.78098739e-01],
         [-2.38379848e-01],
         [ 7.26397572e-01]],

        [[ 3.41181528e-01],
         [ 7.07936984e-01],
         [ 2.87420943e-03],
         [ 2.10625388e-01],
         [ 1.01119456e+00]],

        [[-3.86989282e-01],
         [ 3.37668687e-01],
         [ 1.82818058e-01],
         [-6.33389816e-01],
         [-5.90793602e-01]],

        [[ 1.57722425e-01],
         [ 5.67983774e-01],
         [ 2.07671441e-01],
         [ 7.49728940e-01],
         [-5.77666133e-01]],

        [[-5.97884102e-01],
         [ 9.46973486e-01],
         [ 6.35093312e-01],
         [-4.19587857e-02],
         [ 3.62225855e-01]]],


       [[[ 8.57807287e-01],
         [ 3.41992783e-01],
         [-2.08614346e-01],
         [ 3.59361327e-01],
         [ 3.04932630e-01]],

        [[-3.15655481e-01],
         [-3.42792785e-01],
         [-6.53423262e-02],
       

In [56]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (1,1), activation=tf.nn.elu, input_shape=(5,5,3,)),
    tf.keras.layers.Conv2D(128, (1,1), activation=tf.nn.elu),
    tf.keras.layers.Conv2D(32, (1,1), activation=tf.nn.elu),
    tf.keras.layers.Conv2D(1, (1,1), activation=tf.nn.elu)
])

In [57]:
model(train_data)

<tf.Tensor: shape=(10, 5, 5, 1), dtype=float32, numpy=
array([[[[-0.05334032],
         [-0.05659205],
         [ 0.03190439],
         [ 0.34283587],
         [-0.06109345]],

        [[-0.28196263],
         [-0.10404432],
         [ 0.2533195 ],
         [-0.04562396],
         [-0.22309005]],

        [[-0.15591764],
         [-0.2897539 ],
         [-0.23770314],
         [ 0.10933475],
         [ 0.00936954]],

        [[-0.11108112],
         [-0.20313597],
         [ 0.26310945],
         [ 0.13548249],
         [ 0.12216482]],

        [[ 0.29931784],
         [-0.36872143],
         [ 0.09917687],
         [-0.17857438],
         [-0.16309035]]],


       [[[-0.1409502 ],
         [-0.09266061],
         [ 0.17118774],
         [ 0.06235463],
         [ 0.21831003]],

        [[ 0.183496  ],
         [-0.1606152 ],
         [ 0.34187418],
         [-0.22173744],
         [-0.11282486]],

        [[-0.3001675 ],
         [ 0.13575557],
         [-0.06976408],
         [ 0.3847

In [58]:
loss_object = tf.losses.MeanSquaredError()

In [59]:
def loss(_model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = _model(x, training=training)

  return loss_object(y_true=y, y_pred=y_)

In [60]:
l = loss(model, train_data, train_labels, training=False)
print("Loss test: {}".format(l))

Loss test: 0.46725398302078247


In [61]:
def grad(_model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, training=True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [62]:
optimizer = tf.keras.optimizers.Adam(learning_rate=.01)

In [64]:
train_loss_results = []
train_accuracy_results = []

num_epochs = 20

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    # Training loop - using batches of 32
    for x, y in train_input:
        # Optimize the model
        loss_value, grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Track progress
        epoch_loss_avg.update_state(loss_value)  # Add current batch loss
        # Compare predicted label to actual label
        # training=True is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        epoch_accuracy.update_state(y, model(x, training=True))

    # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    print("Epoch {:03d}: Loss: {:.6f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Epoch 000: Loss: 0.000032, Accuracy: 0.000%
Epoch 001: Loss: 0.000051, Accuracy: 0.000%
Epoch 002: Loss: 0.000078, Accuracy: 0.000%
Epoch 003: Loss: 0.000115, Accuracy: 0.000%
Epoch 004: Loss: 0.000026, Accuracy: 0.000%
Epoch 005: Loss: 0.000102, Accuracy: 0.000%
Epoch 006: Loss: 0.000047, Accuracy: 0.000%
Epoch 007: Loss: 0.000052, Accuracy: 0.000%
Epoch 008: Loss: 0.000105, Accuracy: 0.000%
Epoch 009: Loss: 0.000081, Accuracy: 0.000%
Epoch 010: Loss: 0.000029, Accuracy: 0.000%
Epoch 011: Loss: 0.000062, Accuracy: 0.000%
Epoch 012: Loss: 0.000069, Accuracy: 0.000%
Epoch 013: Loss: 0.008175, Accuracy: 0.000%
Epoch 014: Loss: 0.001247, Accuracy: 0.000%
Epoch 015: Loss: 0.001091, Accuracy: 0.000%
Epoch 016: Loss: 0.000659, Accuracy: 0.000%
Epoch 017: Loss: 0.000193, Accuracy: 0.000%
Epoch 018: Loss: 0.000105, Accuracy: 0.000%
Epoch 019: Loss: 0.000116, Accuracy: 0.000%
